In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [3]:
credits_df = pd.read_csv('dataset/credits.csv')
titles_df = pd.read_csv('dataset/titles.csv')

In [14]:
credits_df
unique_count = credits_df['id'].nunique()
print(unique_count)

5489


In [22]:
titles_df
unique_count = titles_df['id'].nunique()
print(unique_count)


5850


In [32]:
combined_df = pd.merge(titles_df, credits_df, left_on = "id", right_on = "id")
combined_df = combined_df.drop('character', axis=1)
combined_df.dropna(subset=['title'], inplace=True)
combined_df

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,role
0,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,3748,Robert De Niro,ACTOR
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,14658,Jodie Foster,ACTOR
2,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,7064,Albert Brooks,ACTOR
3,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,3739,Harvey Keitel,ACTOR
4,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,48933,Cybill Shepherd,ACTOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77796,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021,NaN,90,['comedy'],['CO'],NaN,tt14585902,3.8,68.0,26.005,6.300,736339,Adelaida Buscato,ACTOR
77797,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021,NaN,90,['comedy'],['CO'],NaN,tt14585902,3.8,68.0,26.005,6.300,399499,Luz Stella Luengas,ACTOR
77798,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021,NaN,90,['comedy'],['CO'],NaN,tt14585902,3.8,68.0,26.005,6.300,373198,Inés Prieto,ACTOR
77799,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021,NaN,90,['comedy'],['CO'],NaN,tt14585902,3.8,68.0,26.005,6.300,378132,Isabel Gaona,ACTOR


In [31]:
# pivot_df = combined_df.pivot_table(index=['id', 'title', 'type', 'description', 'release_year', 'age_certification', 'runtime', 'genres', 'production_countries', 'seasons', 'imdb_id', 'imdb_score', 'imdb_votes', 'tmdb_popularity', 'tmdb_score'],
#                           columns='role',
#                           values='name',
#                           aggfunc=lambda x: ', '.join(x)).reset_index()
# pivot_df
# Pivot the table
pivot_df = combined_df.pivot_table(index=combined_df.index, columns='role', values='name', aggfunc='first')

# Reset index if needed
pivot_df.reset_index(drop=True, inplace=True)
pivot_df

role,ACTOR,DIRECTOR
0,Tamsin Greig,NaN
1,Arsher Ali,NaN
2,Katherine Kelly,NaN
3,Miriam Leone,NaN
4,Stefano Accorsi,NaN
...,...,...
54584,Paul Abell,NaN
54585,Grace Russo,NaN
54586,Camille Marquez,NaN
54587,Irina Aylyarova,NaN


In [33]:
shows = combined_df[combined_df['type'] != 'MOVIE']
shows

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,name,role
163,ts22164,Monty Python's Flying Circus,SHOW,A British sketch comedy series with the shows ...,1969,TV-14,30,"['comedy', 'european']",['GB'],4.0,tt0063929,8.8,73424.0,17.617,8.306,Graham Chapman,ACTOR
164,ts22164,Monty Python's Flying Circus,SHOW,A British sketch comedy series with the shows ...,1969,TV-14,30,"['comedy', 'european']",['GB'],4.0,tt0063929,8.8,73424.0,17.617,8.306,Michael Palin,ACTOR
165,ts22164,Monty Python's Flying Circus,SHOW,A British sketch comedy series with the shows ...,1969,TV-14,30,"['comedy', 'european']",['GB'],4.0,tt0063929,8.8,73424.0,17.617,8.306,Terry Jones,ACTOR
166,ts22164,Monty Python's Flying Circus,SHOW,A British sketch comedy series with the shows ...,1969,TV-14,30,"['comedy', 'european']",['GB'],4.0,tt0063929,8.8,73424.0,17.617,8.306,Eric Idle,ACTOR
167,ts22164,Monty Python's Flying Circus,SHOW,A British sketch comedy series with the shows ...,1969,TV-14,30,"['comedy', 'european']",['GB'],4.0,tt0063929,8.8,73424.0,17.617,8.306,Terry Gilliam,ACTOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77655,ts288861,Let's Eat,SHOW,A food-blogging insurance agent encounters a f...,2021,TV-14,60,"['drama', 'comedy']",['TH'],1.0,tt15978704,8.2,5.0,2.334,NaN,Maneerat Kam-Uan,ACTOR
77656,ts288861,Let's Eat,SHOW,A food-blogging insurance agent encounters a f...,2021,TV-14,60,"['drama', 'comedy']",['TH'],1.0,tt15978704,8.2,5.0,2.334,NaN,Rudklao Amratisha,ACTOR
77657,ts288861,Let's Eat,SHOW,A food-blogging insurance agent encounters a f...,2021,TV-14,60,"['drama', 'comedy']",['TH'],1.0,tt15978704,8.2,5.0,2.334,NaN,Jaturong Mokjok,ACTOR
77658,ts288861,Let's Eat,SHOW,A food-blogging insurance agent encounters a f...,2021,TV-14,60,"['drama', 'comedy']",['TH'],1.0,tt15978704,8.2,5.0,2.334,NaN,Pisamai Wilaisak,ACTOR


TODO: Average score by genre

TODO: Sort Actors by score


In [ ]:

grouped_df = shows.groupby('title').agg(set).reset_index()
grouped_df

,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,character,role
0,#blackAF,{SHOW},{A father takes an irreverent and honest appro...,{2020},{TV-MA},{37},{['comedy']},{['US']},{1.0},{nan},"{nan, nan, nan, nan, nan, nan, nan, nan}","{nan, nan, nan, nan, nan, nan, nan, nan}",{2.924},{7.1},"{1280777, 1060299, 1405871, 834162, 21715, 118...","{Rashida Jones, Ravi Cabot-Conyers, Justin Cla...","{Chloe Barris, Pops Barris, Izzy Barris, Drea ...",{ACTOR}
1,100 Days My Prince,{SHOW},{Lee Yool falls off a cliff and nearly dies in...,{2018},{TV-14},{75},"{['drama', 'thriller', 'comedy', 'action', 'hi...",{['KR']},{1.0},{tt8199972},{7.7},{2068.0},{10.313},{8.323},"{75400, 137871, 948753, 858776, 110122, 296237...","{Kim Seon-ho, Jeong Joon-won, Kim Jae-young, A...","{Goo-Dol, Queen Park, Lord Park Sun-do, Hong-s...","{ACTOR, DIRECTOR}"
2,100 Humans: Life's Questions. Answered.,{SHOW},{One hundred hardy souls from diverse backgrou...,{2020},{TV-14},{37},"{['documentation', 'reality']}",{['US']},{1.0},{tt11823088},{5.7},{4055.0},{3.447},{6.4},"{1922852, 1363723, 1923218, 1120189, 1651806}","{Haley Bockrath, Komalpreet Batth, Braxton Hal...","{Human #64, Human #90, Human #28, Human #84, H...",{ACTOR}
3,13 Reasons Why,{SHOW},"{After a teenage girl's perplexing suicide, a ...",{2017},{TV-MA},{58},"{['drama', 'thriller']}",{['US']},{4.0},{tt1837492},{7.5},{286015.0},{80.574},{7.748},"{282177, 1025665, 112323, 23717, 15238, 119053...","{Jan Luis Castellanos, Miles Heizer, Timothy G...","{Tony Padilla, Tyler Down, Zach Dempsey, Diego...",{ACTOR}
4,13 Reasons Why: Beyond the Reasons,{SHOW},"{Cast members, writers, producers and mental h...",{2017},{TV-MA},{42},{['documentation']},{['US']},{3.0},{tt8615966},{6.2},{328.0},{11.431},{6.5},"{282176, 16431, 1573071, 5144, 1119}","{Katherine Langford, Selena Gomez, Dylan Minne...",{Self},{ACTOR}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1826,revisions,{SHOW},{An isolated city. An unknown enemy. An uncert...,{2019},{TV-MA},{23},"{['animation', 'action', 'scifi']}",{['JP']},{1.0},{tt9522354},{5.9},{612.0},{4.35},{7.0},"{12003, 115686, 3590, 97290, 138, 204, 274, 10...","{Soma Saito, Yoko Hikasa, Yuka Terasaki, Takah...","{Milo, Ryouhei Kuroiwa, Chiharu Isurugi, Chang...",{ACTOR}
1827,นางสาวก้นครัว,{SHOW},"{""Miss Culinary"" reveals the brilliance of a y...",{2019},{TV-14},{0},{[]},{['TH']},{1.0},{nan},"{nan, nan, nan, nan, nan}","{nan, nan, nan, nan, nan}",{2.431},{8.0},"{1850528, 271137, 141199, 163259, 707327}","{Wanida Termthanaporn, Jason Young, Louis Than...",{nan},{ACTOR}
1828,ปริศนาล่าข้ามเวลา,{SHOW},{nan},{2019},{TV-14},{60},{[]},{['TH']},{1.0},{nan},{nan},{nan},{1.473},{7.0},{646657},{Morakot Liu},{nan},{ACTOR}
1829,糖糖Online,{SHOW},"{When a wardrobe malfunction goes viral, a bub...",{2019},{TV-MA},{24},"{['drama', 'family']}",{['TW']},{1.0},{tt11330500},{5.4},{22.0},{0.6},{10.0},"{1260133, 1260135, 1186315, 1260082, 1260020, ...","{Chiao Yuan-Yuan, Jacqueline Zhu Zhi-Ying, Dea...","{Chen Xin-Tian (Candy), Jian Pei-Wen, Zhang Su...",{ACTOR}
